**Importar librerías necesarias**

In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/opt/spark-3.0.0-bin-hadoop2.7"

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.master("local[*]").getOrCreate()
sc=spark.sparkContext

import numpy as np
np.set_printoptions(threshold=sys.maxsize)

**Segmento donde se cambian las palabras del comentario por el índice: (También ordena y agrupa)**

In [4]:
#Arreglo que contiene las palabras del diccionario.
diccionario = (
    np
    .loadtxt('../resultados/100_com/indice_100/part-00000', 
    delimiter= ',', 
    dtype=str)
    )

In [7]:
#RDD con los comentarios de la BD.
#comentarios = np.loadtxt('/home/jsarabia/Documents/IA/analisis_sentimientos/Base/BD_100.csv', delimiter= ',', dtype=str)
comentarios = (
    sc.textFile('../Base/BD_100.csv')
    .map(lambda s: s.split(','))
    .map(lambda col: col[0])
    ).zipWithIndex().filter(lambda col: col[1] > 0).map(lambda col: col[0])
#comentarios.cache()

In [8]:
comentarios.take(8)

['MEXICANOS NO DEJARAN BEBER REFRESCOS PESE AUMENTO HABRA PROXIMO ANO PERO SI ESTAN CAMBIANDO SUS GUSTOS OPCIONES LIGHT SEGUN ESTUDIO DE',
 'SEMAFORO ALERTA ENCUENTRA EN',
 'RUTH RODRIGUEZ ESCRIBE CISMA SALUD LLAMADO MOHAR',
 'MARTIN ESPINOSA ESCRIBE FUERZA ESTADOS',
 'REPORTAN DESAPARICION MAESTRO UAM IZTAPALAPA AYUDANOS COMPARTIR',
 'TERMINA TU DIA BIEN INFORMADO POR',
 'YOUTUBER ENCONTRO NUEVAMENTE AMOR I',
 'A DIEZ MESES INICIO GOBIERNO PRESIDENTE AFIRMO VAMOS BIEN']

In [9]:
def agruparElementos(lista):
    agrupados = []
    cont = 1
    for i in range(len(lista)):
        if(i < len(lista) - 1):
            if(lista[i] == lista[i + 1]):
                cont += 1
            else:
                agrupados.append([lista[i], cont])
                cont = 1
        else:
            agrupados.append([lista[i], cont])
    arr_ag = np.array(agrupados)
    return arr_ag

In [10]:
def llenarArray(lista):
    arr = []
    agrupados = agruparElementos(lista)
    for i in range(1, len(diccionario) + 1):
        if(np.where(agrupados[:, 0] == [i])[0] != None):
            indice = np.where(agrupados[:, 0] == [i])[0]
            repeticiones = agrupados[indice, 1]
            arr.append(str(i) + ':' +str(repeticiones))
        else:
            arr.append(str(i) + ':0')  
    array_numpy = np.array(arr)
    return array_numpy

In [11]:
def sustituir2(comentario):
    #Las palabras por comentario serán referidas en el arreglo palabras.
    palabras = comentario.split(' ')
    lista = []
    for palabra in range(len(palabras)):
        indice = np.where(diccionario == palabras[palabra])[0] + 1
        lista.append(indice)
    lista.sort()
    #El método llenarArray() retorna un arreglo de Numpy.
    arreglo = llenarArray(lista) 
    return arreglo

In [12]:
cad2 = 'MEXICANOS NO DEJARAN CONTENE SELECCIONAIS DE NO BEBER'
arr_prueba = sustituir2(cad2)
#print(arr_prueba)

In [15]:
rdd_agrupados = comentarios.map(lambda s: sustituir2(s))
#rdd_agrupados.take(1)

In [18]:
arr_final = np.array(rdd_agrupados.collect())

In [35]:
def crearIndice():    
    lista = []
    for i in range(100):
        lista.append(i)
    rdd_indice = sc.parallelize(lista)
    return rdd_indice


In [22]:
np.savetxt("../resultados/100_com/cluster.txt", arr_final, fmt='%s')

In [14]:
rdd_agrupados.coalesce(1).saveAsTextFile('/home/jsarabia/Documents/IA/analisis_sentimientos/resultados/palabras_etiquetadas_100')

# Segmento de Prueba